<a href="https://colab.research.google.com/github/ailton-felix/Combinacoes/blob/master/firebase/match_legacy_sings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Selecione o usuário
user = "ailton" #@param ["ailton", "lucas", "joab"] {allow-input: false}

In [ ]:
from google.colab import drive
from google.colab import output
import IPython
import os
import json
drive.mount('/content/drive')

WORK_DIR = '/content/drive/Shareddrives/IA/CIENCIA_DE_DADOS/MATCH_LEGACY_SIGNS_COMMUNITY'
progress_path = os.path.join(WORK_DIR, f'{user}_progress.txt')

# Carregar Progresso
Estrutura salva dos clusters de sinal:

```python
sign_cluster = {
    sign_i: [
        sign_i, sign_j, sign_k, ...
    ]
    sign_l: ...
}
```

In [ ]:
import os

cluster_path = os.path.join(WORK_DIR, f'{user}_sign_cluster.json')

if os.path.exists(cluster_path):
    with open(cluster_path) as f:
        sign_cluster = json.load(f)
else:
    sign_cluster = {}
print(len(sign_cluster), 'clusters de sinal já criado')

In [ ]:
import json
with open(os.path.join(WORK_DIR, 'GROUP_SIGNS_HTA.json')) as f:
    progress_match_signs = json.load(f)[user]
print(len(progress_match_signs), 'grupos de glosa a avaliar')

In [ ]:
def load_progress():
    if os.path.exists(progress_path):
        with open(progress_path) as f:
            progress = int(f.read())
    else:
        progress = 0
        assert input('Progresso não encontrado! Digite "sim" se desejas seguir: ').lower() == 'sim'
        output.clear()

    return progress


def cluster_sign(sign1, sign2):
    has_cluster = False
    for sign_id, signs_list in sign_cluster.items():
        if sign1 in signs_list: 
            if sign2 not in signs_list:
                sign_cluster[sign_id].append(sign2)
                has_cluster = True
                break
        elif sign2 in signs_list:
            if sign1 not in signs_list:
                sign_cluster[sign_id].append(sign1)
                has_cluster = True
                break
    
    if not has_cluster:
        sign_cluster[sign1] = [sign1, sign2]
      
    with open(cluster_path, 'w') as f:
        json.dump(sign_cluster, f, ensure_ascii=False, indent=4)

def remove_sign_from_cluster(sign):
    found = False
    for sign_id, signs_list in sign_cluster.items():
        if sign in signs_list:
            if sign == sign_id:
                signs_list.remove(sign)
                del sign_cluster[sign]
                sign_cluster[signs_list[0]] = signs_list
                found = True
                break
            else:
                signs_list.remove(sign)
                found = True
                break
    if found:
        with open(cluster_path, 'w') as f:
            json.dump(sign_cluster, f, ensure_ascii=False, indent=4)
        print(f'"{cluster_path}" atualizado!')
    else:
        print('sinal não encontrado nos clusters')

progress = load_progress()

# Hora do Match

### Instruções:

1 - Você só precisa executar essa célula uma vez. ("Oxe, mas ela para de executar assim que dou o *run*!" - isso, mas a sua interação será com a página html gerada)

2 - Voto "*No*" errado: execute uma célula como abaixo para retornar ao progresso anterior:

```python
progress = progress - 1
```
e execute essa célula novamente

3 - Voto "*Yes*" errado: faço como a instrução 2 e use a função "*remove_sign_from_cluster(sign)*" para remover o sinal "*sign*" do cluster nos registros:
```python
remove_sign_from_cluster('vzWp4uGIPB08z3Ou0mrK')
```

In [ ]:
html_page = """<script src="https://plugin.handtalk.me/core/2.0.19/hugo.min.js"></script>
    <script>
        var sign1, sign2, hta1, hta2;
        var progressMatchSigns = """ + f'{progress_match_signs}' + """
        var progress = """ + f'{progress}' + """

        function nextMatch() {
            const matchSignsHtas = progressMatchSigns[progress]
            const signs = Object.keys(matchSignsHtas)
            const htas = Object.values(matchSignsHtas)

            sign1 = signs[0]; 
            sign2 = signs[1]; 
            hta1 = htas[0];
            hta2 = htas[1];

            document.getElementById("prog").innerHTML = String(progress+1) + '/' + String(progressMatchSigns.length);
            document.getElementById("sign").innerHTML = sign1 + ' x ' + sign2;

            console.log(`Progresso: ${progress+1}/${progressMatchSigns.length}`);
            console.log(`Sinais: ${sign1} x ${sign2}`);
        }
    </script>
    <div>
        <h1>Usuário: """+ f'{user}' +"""</h2>
        <h2>Progresso: <span id="prog"></span></h2>
        <h3>Sinais: <span id="sign"></span></h2>
    </div>
    <div id="hugos" style="display: flex; flex-direction: row">
        '<div id="hugo1"><canvas width="250" height="400" class="col" style="width: 250px; height: 400px;"></canvas></div>'
        '<div id="hugo2"><canvas width="250" height="400" class="col" style="width: 250px; height: 400px;"></canvas></div>'
    </div>
    <div id="votes" style="display: flex; flex-direction: row">
        <button id='btnYes'>Yes</button>
        <button id='btnNo'>No</button>
        <button id='btnRun'>Run Animation</button>
    </div>
    <script>
        
        function addHugo(hugoId) {
            var hugo = new Hugo("6b1fa06c035e881027a8050a0fd971ac");
            hugo.load().then(function (elem) {
            hugo.resize(250, 400);
            elem.setAttribute("class", "col");
            document.querySelector(hugoId).appendChild(elem);
            }).catch(function (error) {
                errorMsgElem.innerText = error;
            });
            return hugo
        }
        nextMatch();
    </script>
    <script>
        var hugo1 = addHugo("#hugo1")
        var hugo2 = addHugo("#hugo2")
    </script>
    <script>
        function executeAnim(anim1, anim2) {
            hugo1.prepareSequences([anim1.split('$')])
                .then(hugo1.signAnimationsCodes);
            hugo2.prepareSequences([anim2.split('$')])
            .then(hugo2.signAnimationsCodes);
        }
    </script>
    <script>
        executeAnim(hta1, hta2);

        document.querySelector('#btnRun').onclick = () => {
            executeAnim(hta1, hta2);
        };

        document.querySelector('#btnYes').onclick = () => {
            google.colab.kernel.invokeFunction('notebook.voteYes', [], {});
            progress++;
            nextMatch();
            executeAnim(hta1, hta2);
        };

        document.querySelector('#btnNo').onclick = () => {
            google.colab.kernel.invokeFunction('notebook.voteNo', [], {});
            progress++;
            nextMatch();
            executeAnim(hta1, hta2);
        };
    </script>"""


def register_progress():
    global progress
    progress+=1
    with open(progress_path, 'w') as f:
        f.write(str(progress))


def vote_yes():
    #   criar voto entre os segmentos dos sinais -> usar 'sign_segment_dic'
    #   agrupar sinais
    global progress_match_signs, progress
    match_signs_htas = progress_match_signs[progress]
    
    signs = list(match_signs_htas.keys())
    sign1, sign2 = signs[0], signs[1]

    cluster_sign(sign1, sign2)

    # registrar progresso
    register_progress()


def vote_no():
    global progress
    # registrar progresso
    register_progress()


progress = load_progress()
output.register_callback('notebook.voteYes', vote_yes)
output.register_callback('notebook.voteNo', vote_no)

display(IPython.display.HTML(html_page))

In [ ]:
# no final substituir os antigos sinais pelo novo sinal do cluster, baseado no seu segmento candidato (deixe que o Tio Ailton vai fazer isso quando tudo acabar)